# ReAct agent basic

- ReAct 가 어떻게 동작하는지 가장 간단한 예제로 설명

In [1]:
import os
import sys
import logging

from pydantic import Field
from dotenv import load_dotenv
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
from llama_index.core.tools.types import BaseTool
from llama_index.llms.bedrock_converse import BedrockConverse

In [2]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
handler = logging.StreamHandler(sys.stdout)
handler.setFormatter(formatter)
logger.addHandler(handler)

load_dotenv()

True

In [3]:
MODEL_ID=os.environ.get("MODEL_ID", "anthropic.claude-3-haiku-20240307-v1:0")
AWS_PROFILE_NAME=os.environ.get("AWS_PROFILE_NAME", None)

## Setup tools

In [4]:
def get_weather(
    date: str = Field(
        description="A date, formmated in RFC3999",
    ),
    location: str = Field(
        description="A city name and state, formatted like '<name>, <state>'"
    ),
) -> str:
    """
    Returns:
        (str): Useful for getting the weather for a given location.
    """
    return f"Weather in {location}, {date} is rainy."

In [5]:
# setup tools
tools: list[BaseTool] = [
    FunctionTool.from_defaults(get_weather),
]

In [6]:
print(tools[0].metadata.description)

get_weather(date: str = FieldInfo(annotation=NoneType, required=True, description='A date, formmated in RFC3999'), location: str = FieldInfo(annotation=NoneType, required=True, description="A city name and state, formatted like '<name>, <state>'")) -> str

    Returns:
        (str): Useful for getting the weather for a given location.
    


## Setup Agent

In [7]:
# setup llm
llm = BedrockConverse(
    model=MODEL_ID,
    max_tokens=1024 * 2,
    temperature=0.4,
    profile_name=AWS_PROFILE_NAME,
)

# setup agent
agent = ReActAgent.from_tools(
    tools=tools,
    llm=llm,
    verbose=True,
)

### Agent prompt

https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/agent/react/templates/system_header_template.md

In [8]:
print(agent.get_prompts()['agent_worker:system_prompt'].template)

You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}


## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

NEVER surround your response with markdown code markers. You may use code markers within your response if y

## Invoke agent with tool

In [9]:
resp = agent.chat(
    "2024-10-18 수원 날씨는 어때?"
)

> Running step 30e9de6f-944f-4ede-841f-e773841757fc. Step input: 2024-10-18 수원 날씨는 어때?
Thought: The current language of the user is: Korean. I need to use a tool to help me answer the question.
Action: get_weather
Action Input: {'date': '2024-10-18', 'location': '수원'}
Observation: Weather in 수원, 2024-10-18 is rainy.
> Running step 9af41b1a-3fbc-4f53-ba42-992a1282e8b1. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: 2024년 10월 18일 수원의 날씨는 비가 올 것으로 예상됩니다.
